In [1]:
import os

In [2]:
%pwd

'd:\\LoanTap\\LoanTap\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransormationConfig:
    root_dir: Path
    data_path: Path
    ohencoder_path: Path
    le_grade_path:  Path
    le_subgrade_path: Path
    le_emp_length_path: Path



In [5]:
from src.LoanTapPred.constants import *
from src.LoanTapPred.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransormationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransormationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            ohencoder_path=config.ohencoder_path,
            le_grade_path=config.le_grade_path,
            le_subgrade_path=config.le_subgrade_path,
            le_emp_length_path=config.le_emp_length_path,
        )

        return data_transformation_config


In [6]:
import os
from src.LoanTapPred import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from src.LoanTapPred.utils.data_transformation_utils import *
from src.LoanTapPred.utils.common import save_bin
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [7]:
class DataTransormation:
    def __init__(self, config: DataTransormationConfig):
        self.config = config

    def transform(self):
        try:
            data = pd.read_csv(self.config.data_path)
            data = missing_value_imputation(data)
            data = feature_engineering(data)
            data = nominal_feature_encoding(data, self.config.ohencoder_path)
            data = ordinal_feature_encoding(data, ordinal_features=['grade', 'sub_grade', 'emp_length'], 
                                            paths=[self.config.le_grade_path, self.config.le_subgrade_path,self.config.le_emp_length_path])
            data = drop_collinear_features(data)
            return data
        except Exception as e:
            raise e
        
    def train_test_splitting(self):
        data = self.transform()
        train,test = train_test_split(data, test_size=0.20)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)
        logger.info("Train and test files created")
        print(train.shape, test.shape)    




In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransormation(config=data_transformation_config) 
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2025-01-22 23:07:09,831: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-01-22 23:07:09,854: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 23:07:09,877: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-22 23:07:09,879: INFO: common: created directory at: artifacts]
[2025-01-22 23:07:09,882: INFO: common: created directory at: artifacts/data_transformation]


KeyError: "['le_subgrade'] not found in axis"